In [63]:
import libstempo as lst
import libstempo.toasim as toasim
import libstempo.plot as lstplot
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sys
import enterprise_GWecc as gwecc
import ephem
import json

In [64]:
data_dir = "BayesHopper_sims/"
par_fmt = "ParFiles/{}.par"
tim_fmt = "TimFiles/fake_{}_study4_highergwb.tim"

output_dir = "output/"

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [65]:
# Get list of pulsars
psrjs = sorted([os.path.basename(p).split('.')[0] for p in glob.glob(data_dir + par_fmt.format("*"))])
#print(psrjs)

In [66]:
def read_pulsar(jname):
    psr_dir = data_dir 
    par_file = psr_dir + "/" + par_fmt.format(jname)
    tim_file = psr_dir + "/" + tim_fmt.format(jname)
    
    psr = lst.tempopulsar(parfile=par_file, timfile=tim_file)
    
    return psr

psrs = []
for psrj in psrjs:
    psr = read_pulsar(psrj)
    psrs.append(psr)

021552      -0.064841     Y
PMDEC (mas/yr)  -5.03376865001804         -4.74938014908657         0.084489      0.28439       Y
PX (mas)        4.02291243326134          4.51389242590857          0.10735       0.49098       Y
TZRMJD          -nan                      53000.0000000009          0             -nan          N
TZRFRQ (MHz)    -nan                      1440                      0             -nan          N
TZRSITE         AXIS                     
TRES            -nan                      42.3183252976181          0             -nan          N
EPHVER          TEMPO2                    TEMPO2                    0             0             N
DM_SERIES       TAYLOR                   
---------------------------------------------------------------------------------------------------
[textOutput.C:299] Notice: Parameter uncertainties NOT multiplied by sqrt(red. chisq)


Derived parameters:

P0 (s)      = 0.00486545328281111       9.7125e-18   
P1          = 1.01709945714169e-20   

In [67]:
#for psr in psrs:
#    print(psr.name)

In [68]:
def uniform_sample(n,a,b):
    return np.random.rand(n)*(b-a) + a

#efac_min, efac_max = 0.8, 1.2
#efacs = uniform_sample(len(psrs), efac_min, efac_max)

#log10_equad_min, log10_equad_max = -10, -7
#log10_equads = uniform_sample(len(psrs), log10_equad_min, log10_equad_max)
#equads = 10**log10_equads

#log10_rn_A_min, log10_rn_A_max = -20, -15
#log10_rn_As = uniform_sample(len(psrs), log10_rn_A_min, log10_rn_A_max)
#rn_As = 10**log10_rn_As

#rn_gamma_min, rn_gamma_max = 0.1, 7
#rn_gammas = uniform_sample(len(psrs), rn_gamma_min, rn_gamma_max)

#rn_nharms = 15

In [69]:
#true_noise_params = np.transpose([psrjs, efacs, equads, rn_As, rn_gammas])
#header = "PSRJ\tEFAC\tEQUAD\tRN_AMPL\tRN_GAMMA"
#np.savetxt(output_dir + "/true_noise_params.dat", true_noise_params, fmt='%s %s %s %s %s', 
#           comments='#', header=header)

In [70]:
savedir = "GWB-highred/"
if not os.path.exists(output_dir+savedir):
    os.mkdir(output_dir+savedir)

In [71]:
gwb_Amp = 1e-13
gwb_gam = 13./3
true_gwb_params = np.array([[gwb_Amp, gwb_gam]])
np.savetxt(output_dir + savedir + "/true_gwb_params.dat", true_gwb_params,
           comments='#', header="Amp\tgamma")

In [72]:
red_Amp = 1e-12
red_gam = 3
true_red_params = np.array([[red_Amp, red_gam, 15]])
np.savetxt(output_dir + savedir + "/true_red_params.dat", true_red_params,
           comments='#', header="Amp\tgamma\tNharm")

In [73]:
#for psr, ef, eq, rnA, rngam in zip(psrs, efacs, equads, rn_As, rn_gammas):
for psr in psrs:
    toasim.make_ideal(psr)
    toasim.add_efac(psr, 1)
    #toasim.add_equad(psr, eq)
    toasim.add_rednoise(psr, red_Amp, red_gam, components=15)
    print("Simulated TOAs for", psr.name)
toasim.createGWB(psrs, Amp=gwb_Amp, gam=gwb_gam)

Simulated TOAs for JPSR00
Simulated TOAs for JPSR01
Simulated TOAs for JPSR02
Simulated TOAs for JPSR03
Simulated TOAs for JPSR04
Simulated TOAs for JPSR05
Simulated TOAs for JPSR06
Simulated TOAs for JPSR07
Simulated TOAs for JPSR08
Simulated TOAs for JPSR09
Simulated TOAs for JPSR10
Simulated TOAs for JPSR11
Simulated TOAs for JPSR12
Simulated TOAs for JPSR13
Simulated TOAs for JPSR14
Simulated TOAs for JPSR15
Simulated TOAs for JPSR16
Simulated TOAs for JPSR17
Simulated TOAs for JPSR18
Simulated TOAs for JPSR19


In [74]:
def save_psr_sim(psr,savedir):
    if not os.path.exists(f"{output_dir}/{savedir}/"):
        os.mkdir(f"{output_dir}/{savedir}/")
        
    print("Writing simulated data for", psr.name)
    psr.savepar("{}/{}/{}_simulate.par".format(output_dir, savedir, psr.name))
    psr.savetim("{}/{}/{}_simulate.tim".format(output_dir, savedir, psr.name))
    lst.purgetim("{}/{}/{}_simulate.tim".format(output_dir, savedir, psr.name))
    
for psr in psrs:
    psr.fit()
    save_psr_sim(psr, savedir)

Writing simulated data for JPSR00
Writing simulated data for JPSR01


Results for PSR JPSR00


RMS pre-fit residual = 0.000 (us), RMS post-fit residual = 37.994 (us)
Fit Chisq = 0	Chisqr/nfree = 0.00/0 = -nan	pre/post = 0
Number of fit parameters: 0
Number of points in fit = 0
Offset: 0 1 offset_e*sqrt(n) = 0 n = 0


PARAMETER       Pre-fit                   Post-fit                  Uncertainty   Difference   Fit
---------------------------------------------------------------------------------------------------
RAJ (rad)       2.44266464632084          2.44266466246464          1.1028e-09    1.6144e-08    Y
RAJ (hms)       9:19:49.05                 09:19:49.0500000         1.5165e-05    0.00022199   
DECJ (rad)      -1.32138538298914         -1.32138537260125         2.65e-10      1.0388e-08    Y
DECJ (dms)      -75:42:35.3               -75:42:35.30000           5.466e-05     0.0021427    
F0 (s^-1)       205.530696088273          205.530696088331          4.103e-13     5.835e-11   

BiWriting simulated data for JPSR04
Writing simulated data for JPSR05
nary model           NONE
[textOutput.C:1086] Warning: UNITS was not set in the parameter file: using TCB (tempo2)
In here writing a new parameter file: output//GWB-highred//JPSR03_simulate.par
Notice: There were 1 warnings. Summaries are shown below, check logs for full details.
Warning #1: UNITS was not set in the parameter file: using TCB (tempo2)


Results for PSR JPSR04


RMS pre-fit residual = 0.000 (us), RMS post-fit residual = 48.456 (us)
Fit Chisq = 0	Chisqr/nfree = 0.00/0 = -nan	pre/post = 0
Number of fit parameters: 0
Number of points in fit = 0
Offset: 0 1 offset_e*sqrt(n) = 0 n = 0


PARAMETER       Pre-fit                   Post-fit                  Uncertainty   Difference   Fit
---------------------------------------------------------------------------------------------------
RAJ (rad)       5.18903985354538          5.18903983264217          4.161e-10     -2.0903e-08   Y
RAJ (hms)       19:49:14.42  

bs (G)      = 2Writing simulated data for JPSR08
Writing simulated data for JPSR09
.2625e+08

Total proper motion = 11.953 +/- 0.079753 mas/yr
Total time span = 3630.000 days = 9.938 years

Tempo2 usage
Units:                 TCB (tempo2)
Time ephemeris:        IF99 (tempo2)
Troposphere corr.?     Yes (tempo2)
Dilate freq?           Yes (tempo2)
Electron density (1AU) 4
Solar system ephem     DE436
Time scale             TT(BIPM2016)
Binary model           NONE
[textOutput.C:1086] Warning: UNITS was not set in the parameter file: using TCB (tempo2)
In here writing a new parameter file: output//GWB-highred//JPSR07_simulate.par
Notice: There were 1 warnings. Summaries are shown below, check logs for full details.
Warning #1: UNITS was not set in the parameter file: using TCB (tempo2)


Results for PSR JPSR08


RMS pre-fit residual = 0.000 (us), RMS post-fit residual = 74.663 (us)
Fit Chisq = 0	Chisqr/nfree = 0.00/0 = -nan	pre/post = 0
Number of fit parameters: 0
Number of points in fit =

DM_SERIES       TAYLOR               Writing simulated data for JPSR12
    
---------------------------------------------------------------------------------------------------
[textOutput.C:299] Notice: Parameter uncertainties NOT multiplied by sqrt(red. chisq)


Derived parameters:

P0 (s)      = 0.00486545328280588       9.7124e-18   
P1          = 1.01413227377464e-20      6.0022e-26   
tau_c (Myr) = 7606.6
bs (G)      = 2.2478e+08

Total proper motion = 5.5423 +/- 0.011855 mas/yr
Total time span = 3630.000 days = 9.938 years

Tempo2 usage
Units:                 TCB (tempo2)
Time ephemeris:        IF99 (tempo2)
Troposphere corr.?     Yes (tempo2)
Dilate freq?           Yes (tempo2)
Electron density (1AU) 4
Solar system ephem     DE436
Time scale             TT(BIPM2016)
Binary model           NONE
[textOutput.C:1086] Warning: UNITS was not set in the parameter file: using TCB (tempo2)
In here writing a new parameter file: output//GWB-highred//JPSR11_simulate.par
Notice: There were 1

TZRFRQ (MHz)    Writing simulated data for JPSR16
-nan                      1440                      0             -nan          N
TZRSITE         AXIS                     
TRES            -nan                      149.294043992986          0             -nan          N
EPHVER          TEMPO2                    TEMPO2                    0             0             N
DM_SERIES       TAYLOR                   
---------------------------------------------------------------------------------------------------
[textOutput.C:299] Notice: Parameter uncertainties NOT multiplied by sqrt(red. chisq)


Derived parameters:

P0 (s)      = 0.00486545328279273       9.7134e-18   
P1          = 1.02254249274205e-20      6.0025e-26   
tau_c (Myr) = 7544.1
bs (G)      = 2.2571e+08

Total proper motion = 6.8861 +/- 0.074875 mas/yr
Total time span = 3630.000 days = 9.938 years

Tempo2 usage
Units:                 TCB (tempo2)
Time ephemeris:        IF99 (tempo2)
Troposphere corr.?     Yes (tempo2)
Dilate

In [ ]:
day_to_s = 24*3600

tref = float( max([max(psr.toas()) for psr in psrs]) * day_to_s )

gwecc_params = {"cos_gwtheta" : 0.3,
                "gwphi"       : np.pi/4,
                "log10_dist"  : -1.2,
                "log10_h"     : None,
                "psi"         : 0, 
                "cos_inc"     : 0.6,
                "log10_M"     : 9, 
                "q"           : 1, 
                "log10_F"     : -8, 
                "e0"          : 0.5, 
                "gamma0"      : 0, 
                "l0"          : 0, 
                "tref"        : tref,
                "z"           : 0.0}

def add_gwecc(psr, gwecc_params):
    toas = (psr.toas() * day_to_s).astype("double")
    
    params = dict(zip(psr.pars(), psr.vals()))
    if 'RAJ' in psr.pars():
        theta = float(np.pi/2 - params['DECJ'])
        phi = float(params['RAJ'])
    elif "ELAT" in psr.pars():
        rad_to_deg = 180.0 / np.pi
        coords = ephem.Equatorial(ephem.Ecliptic(str(psr["ELONG"].val * rad_to_deg), str(psr["ELAT"].val * rad_to_deg)))

        theta = np.pi / 2 - float(repr(coords.dec))
        phi = float(repr(coords.ra))
    
    signal = gwecc.eccentric_cw_delay(toas=toas, theta=theta, phi=phi, 
                                      pdist=1, 
                                      psrTerm=False, evolve=True,
                                      **gwecc_params) / day_to_s
    
    psr.stoas[:] += signal 
    
    return signal

In [ ]:
with open(output_dir + "/true_gwecc_params.dat", 'w') as outfile:
    json.dump(gwecc_params, outfile, indent=4)

In [ ]:
for psr in psrs:
    signal = add_gwecc(psr, gwecc_params)
    print("Simulated TOAs for", psr.name)

In [ ]:
for psr, ef, eq, rnA, rngam in zip(psrs, efacs, equads, rn_As, rn_gammas):
    print(psr.name, ef, eq, rnA, rngam)
    lstplot.plotres(psr)
    plt.show()

In [ ]:
for psr in psrs:
    psr.fit()
    save_psr_sim(psr, "GWB+GWecc(E)")